In [123]:
import pandas as pd
from scipy.stats import entropy
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter



In [128]:

def get_bar_info(bar_visit,output):
    visit_201901 = pd.read_csv(bar_visit)
    census = pd.read_csv(r'C:\Users\dzhang373\Desktop\GIS HW\advGIS-finalproject\data\raw\census\acs_ATLtract_2019.csv')
    census2 = census.loc[census.mhi>0,:]
    bar_range = pd.read_csv(r'C:\Users\dzhang373\Desktop\GIS HW\advGIS-finalproject\data\cleaned\Servicearea_tract.csv')
    bar_filter = np.unique(bar_range['placekey'])
    
    dict_201901 = {}
    unique_bar_201901 = []
    
    for idx, row in visit_201901.iterrows():
        if row['placekey'] not in unique_bar_201901:
            unique_bar_201901.append(row['placekey'])
            
            
    for bar in unique_bar_201901:
        
        bar_selected = visit_201901.loc[visit_201901['placekey'] == bar]
        bar_black_ratio = 0
        bar_white_ratio = 0
        bar_asian_ratio = 0
        bar_native_ratio = 0
        bar_mhi_mean = 0
        bar_n_visitor_weight = 0
        race_entropy = 0
        weighted_var = 0

        bar_mhi_tmp = []
        bar_weights = []
        

        for idx, row in bar_selected.iterrows():

            if str(row['FIPS'])[0:2] == '13' and len(str(row['FIPS'])) == 11 and census2.loc[census2['FIPS'] == int(row['FIPS'])].empty == False:
                bar_n_visitor_weight += int(row['n_visitor'])

                bar_black_ratio += float(census2.loc[census2['FIPS'] == float(row['FIPS'])]['P_black']) * int(row['n_visitor'])
                bar_white_ratio += float(census2.loc[census2['FIPS'] == float(row['FIPS'])]['P_white']) * int(row['n_visitor'])
                bar_asian_ratio += float(census2.loc[census2['FIPS'] == float(row['FIPS'])]['P_asian']) * int(row['n_visitor'])
                bar_native_ratio += float(census2.loc[census2['FIPS'] == float(row['FIPS'])]['P_native']) * int(row['n_visitor'])

                bar_mhi_mean += int(census2.loc[census2['FIPS'] == int(row['FIPS'])]['mhi']) * int(row['n_visitor'])

                bar_mhi_tmp.append(int(census2.loc[census2['FIPS'] == int(row['FIPS'])]['mhi']))
                bar_weights.append(int(row['n_visitor']))

        bar_race_ratio_sum = bar_black_ratio + bar_white_ratio + bar_asian_ratio + bar_native_ratio        
        if bar_n_visitor_weight != 0 and bar_race_ratio_sum != 0:
            bar_black_ratio /= bar_n_visitor_weight 
            bar_white_ratio /= bar_n_visitor_weight 
            bar_asian_ratio /= bar_n_visitor_weight 
            bar_native_ratio /= bar_n_visitor_weight 

            bar_mhi_mean /= bar_n_visitor_weight

            weighted_var = np.sum(np.array(bar_weights) * (np.array(bar_mhi_tmp) - bar_mhi_mean)**2) / np.sum(bar_weights)


            bar_black_ratio /= bar_race_ratio_sum
            bar_white_ratio /= bar_race_ratio_sum
            bar_asian_ratio /= bar_race_ratio_sum
            bar_native_ratio /= bar_race_ratio_sum
            race_entropy = entropy([bar_black_ratio,bar_white_ratio,bar_asian_ratio,bar_native_ratio], base=2)

        dict_201901[bar] = [race_entropy,bar_mhi_mean,weighted_var]
        
    result = [x for x in list(dict_201901.keys()) if x not in bar_filter]

    for i in result:
        del dict_201901[i]
    df = pd.DataFrame()
    df['placekey'] = list(dict_tmp.keys())
    df['info'] = list(dict_tmp.values())
    df[['customer_race_entropy','customer_bar_mhi_mean','customer_weighted_var']] = df['info'].apply(pd.Series)
    
    df = df.drop('info', axis=1)
    df.to_csv(output, index=False)
    
#     return dict_201901
    


In [139]:
bar_visit = r'C:\Users\dzhang373\Desktop\GIS HW\advGIS-finalproject\data\raw\SafeGraph\mpatterns_us_naics722410_201912\mpatterns_us_naics722410_201912.csv'
output = 'bar_201912.csv'

get_bar_info(bar_visit,output)

In [115]:
df = pd.DataFrame()

In [120]:
df['placekey'] = list(dict_tmp.keys())
df['info'] = list(dict_tmp.values())
df[['customer_race_entropy','customer_bar_mhi_mean','customer_weighted_var']] = df['info'].apply(pd.Series)

In [122]:

df = df.drop('info', axis=1)
df
df.to_csv('my_data.csv', index=False)

,placekey,customer_race_entropy,customer_bar_mhi_mean,customer_weighted_var
0,zzy-223@8g7-37t-nbk,1.267624,57420.333333,4.115851e+07
1,22c-222@8g6-yvg-3kf,1.352494,75855.968421,1.159568e+09
2,222-222@8gb-7gv-3qz,1.004833,83425.750000,5.542198e+08
3,222-222@8g7-484-hqz,1.370757,84299.000000,1.235665e+09
4,224-222@8gb-px9-8n5,0.775613,102196.964706,5.350868e+08
...,...,...,...,...
584,222-223@8g6-y4q-7yv,0.930714,109866.800000,1.244795e+09
585,223-222@8g7-3dy-f4v,1.196580,66492.796296,2.752740e+08
586,zzw-225@8g6-yjj-x3q,1.440598,61059.360000,5.222728e+08
587,228-222@8g6-x3p-cbk,0.817824,64156.090909,1.809708e+08


In [62]:
visit_201901 = pd.read_csv(r'C:\Users\dzhang373\Desktop\GIS HW\advGIS-finalproject\data\raw\SafeGraph\mpatterns_us_naics722410_201901\mpatterns_us_naics722410_201901.csv')


In [63]:
visit_201901

,placekey,raw_visit_counts,raw_visitor_counts,visits_by_day,median_dwell,bucketed_dwell_times,popularity_by_hour,normalized_visits_by_state_scaling,normalized_visits_by_region_naics_visits,normalized_visits_by_region_naics_visitors,time,FIPS,n_visitor
0,zzy-223@8g7-37t-nbk,40.0,30.0,"[0,3,3,1,1,1,1,0,0,0,0,0,3,0,0,1,3,2,3,3,1,2,2...",19.0,"{""<5"":0,""5-10"":16,""11-20"":6,""21-60"":4,""61-120""...","[13,14,13,13,17,16,13,12,9,8,8,5,6,9,5,4,3,3,3...",461.937828,0.000114,0.000183,201901,13063040609,4
1,zzy-223@8g7-37t-nbk,40.0,30.0,"[0,3,3,1,1,1,1,0,0,0,0,0,3,0,0,1,3,2,3,3,1,2,2...",19.0,"{""<5"":0,""5-10"":16,""11-20"":6,""21-60"":4,""61-120""...","[13,14,13,13,17,16,13,12,9,8,8,5,6,9,5,4,3,3,3...",461.937828,0.000114,0.000183,201901,13151070304,4
2,zzy-223@8g7-37t-nbk,40.0,30.0,"[0,3,3,1,1,1,1,0,0,0,0,0,3,0,0,1,3,2,3,3,1,2,2...",19.0,"{""<5"":0,""5-10"":16,""11-20"":6,""21-60"":4,""61-120""...","[13,14,13,13,17,16,13,12,9,8,8,5,6,9,5,4,3,3,3...",461.937828,0.000114,0.000183,201901,45075011000,4
3,zzy-223@8g7-37t-nbk,40.0,30.0,"[0,3,3,1,1,1,1,0,0,0,0,0,3,0,0,1,3,2,3,3,1,2,2...",19.0,"{""<5"":0,""5-10"":16,""11-20"":6,""21-60"":4,""61-120""...","[13,14,13,13,17,16,13,12,9,8,8,5,6,9,5,4,3,3,3...",461.937828,0.000114,0.000183,201901,13215010501,4
4,zzy-223@8g7-37t-nbk,40.0,30.0,"[0,3,3,1,1,1,1,0,0,0,0,0,3,0,0,1,3,2,3,3,1,2,2...",19.0,"{""<5"":0,""5-10"":16,""11-20"":6,""21-60"":4,""61-120""...","[13,14,13,13,17,16,13,12,9,8,8,5,6,9,5,4,3,3,3...",461.937828,0.000114,0.000183,201901,13151070404,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
28839,224-222@8gd-q4k-jjv,103.0,50.0,"[5,2,1,4,6,0,0,3,4,2,4,8,0,4,5,3,3,3,4,0,1,2,6...",60.0,"{""<5"":0,""5-10"":15,""11-20"":14,""21-60"":23,""61-12...","[23,23,16,10,0,0,0,0,2,2,1,8,10,15,11,16,15,17...",1189.489908,0.000294,0.000471,201901,13051003302,4
28840,224-222@8gd-q4k-jjv,103.0,50.0,"[5,2,1,4,6,0,0,3,4,2,4,8,0,4,5,3,3,3,4,0,1,2,6...",60.0,"{""<5"":0,""5-10"":15,""11-20"":14,""21-60"":23,""61-12...","[23,23,16,10,0,0,0,0,2,2,1,8,10,15,11,16,15,17...",1189.489908,0.000294,0.000471,201901,13051010501,4
28841,224-222@8gd-q4k-jjv,103.0,50.0,"[5,2,1,4,6,0,0,3,4,2,4,8,0,4,5,3,3,3,4,0,1,2,6...",60.0,"{""<5"":0,""5-10"":15,""11-20"":14,""21-60"":23,""61-12...","[23,23,16,10,0,0,0,0,2,2,1,8,10,15,11,16,15,17...",1189.489908,0.000294,0.000471,201901,13051010809,4
28842,224-222@8gd-q4k-jjv,103.0,50.0,"[5,2,1,4,6,0,0,3,4,2,4,8,0,4,5,3,3,3,4,0,1,2,6...",60.0,"{""<5"":0,""5-10"":15,""11-20"":14,""21-60"":23,""61-12...","[23,23,16,10,0,0,0,0,2,2,1,8,10,15,11,16,15,17...",1189.489908,0.000294,0.000471,201901,13051004209,4


In [64]:
census = pd.read_csv(r'C:\Users\dzhang373\Desktop\GIS HW\advGIS-finalproject\data\raw\census\acs_ATLtract_2019.csv')

# Drop columns with True values in mask
census2 = census.loc[census.mhi>0,:]


In [65]:
census2

,mhi,pop,pop_white,pop_black,pop_asian,pop_native,STATEFP,COUNTYFP,TRACTCE,FIPS,P_white,P_black,P_asian,P_native
0,64571.0,13838.0,3906.0,7869.0,329.0,181.0,13,135,50709,13135050709,0.282266,0.568652,0.023775,0.013080
1,49054.0,9711.0,3847.0,2352.0,69.0,956.0,13,135,50314,13135050314,0.396149,0.242200,0.007105,0.098445
2,53693.0,6361.0,2341.0,2342.0,226.0,226.0,13,135,50315,13135050315,0.368024,0.368181,0.035529,0.035529
3,75036.0,4072.0,1783.0,2133.0,3.0,48.0,13,135,50415,13135050415,0.437868,0.523821,0.000737,0.011788
4,102500.0,5592.0,3330.0,824.0,9.0,827.0,13,135,50425,13135050425,0.595494,0.147353,0.001609,0.147890
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
946,81944.0,5424.0,4225.0,1111.0,0.0,0.0,13,223,120604,13223120604,0.778945,0.204830,0.000000,0.000000
947,72316.0,11339.0,5739.0,3766.0,0.0,399.0,13,151,70205,13151070205,0.506129,0.332128,0.000000,0.035188
948,96082.0,7661.0,2378.0,4216.0,0.0,847.0,13,151,70309,13151070309,0.310403,0.550320,0.000000,0.110560
949,77692.0,14560.0,7765.0,6667.0,0.0,63.0,13,151,70502,13151070502,0.533310,0.457898,0.000000,0.004327


In [82]:
bar_range = pd.read_csv(r'C:\Users\dzhang373\Desktop\GIS HW\advGIS-finalproject\data\cleaned\Servicearea_tract.csv')
bar_filter = np.unique(bar_range['placekey'])
bar_filter

array(['222-222@8g6-x6q-2zf', '222-222@8g6-x74-249',
       '222-222@8g6-x75-y5f', '222-222@8g6-x78-pqf',
       '222-222@8g6-x8s-rtv', '222-222@8g6-xhs-xqz',
       '222-222@8g6-xr8-bp9', '222-222@8g6-y3w-y5f',
       '222-222@8g6-y44-435', '222-222@8g6-y4w-4sq',
       '222-222@8g6-yc4-jsq', '222-222@8g6-ycf-gx5',
       '222-222@8g6-yd5-r6k', '222-222@8g6-yhk-59f',
       '222-222@8g6-yjh-td9', '222-222@8g6-yjh-wtv',
       '222-222@8g6-yjx-49z', '222-222@8g6-ykr-5j9',
       '222-222@8g6-ykz-6p9', '222-222@8g6-ykz-9mk',
       '222-222@8g6-ykz-qfz', '222-222@8g6-ykz-zfz',
       '222-222@8g6-ym2-f2k', '222-222@8g6-ym4-mff',
       '222-222@8g6-ym4-pjv', '222-222@8g6-ym4-yn5',
       '222-222@8g6-ym4-yvz', '222-222@8g6-ym5-249',
       '222-222@8g6-ym5-59f', '222-222@8g6-ym5-rc5',
       '222-222@8g6-ym8-dgk', '222-222@8g6-ymt-z4v',
       '222-222@8g6-yqm-6rk', '222-222@8g6-ytb-j5f',
       '222-222@8g6-ytk-mc5', '222-222@8g6-ytr-5cq',
       '222-222@8g7-223-mhq', '222-222@8g7-2k6

In [66]:
dict_201901 = {}
unique_bar_201901 = []


In [67]:
for idx, row in visit_201901.iterrows():
    if row['placekey'] not in unique_bar_201901:
        unique_bar_201901.append(row['placekey'])
        


In [68]:
for bar in unique_bar_201901:
    bar_selected = visit_201901.loc[visit_201901['placekey'] == bar]
    bar_black_ratio = 0
    bar_white_ratio = 0
    bar_asian_ratio = 0
    bar_native_ratio = 0
    bar_mhi_mean = 0
    bar_n_visitor_weight = 0
    race_entropy = 0
    weighted_var = 0
    
    bar_mhi_tmp = []
    bar_weights = []
    
    for idx, row in bar_selected.iterrows():
        
        if str(row['FIPS'])[0:2] == '13' and len(row['FIPS']) == 11 and census2.loc[census2['FIPS'] == int(row['FIPS'])].empty == False:
            bar_n_visitor_weight += int(row['n_visitor'])
            
            bar_black_ratio += float(census2.loc[census2['FIPS'] == float(row['FIPS'])]['P_black']) * int(row['n_visitor'])
            bar_white_ratio += float(census2.loc[census2['FIPS'] == float(row['FIPS'])]['P_white']) * int(row['n_visitor'])
            bar_asian_ratio += float(census2.loc[census2['FIPS'] == float(row['FIPS'])]['P_asian']) * int(row['n_visitor'])
            bar_native_ratio += float(census2.loc[census2['FIPS'] == float(row['FIPS'])]['P_native']) * int(row['n_visitor'])

            bar_mhi_mean += int(census2.loc[census2['FIPS'] == int(row['FIPS'])]['mhi']) * int(row['n_visitor'])
            
            bar_mhi_tmp.append(int(census2.loc[census2['FIPS'] == int(row['FIPS'])]['mhi']))
            bar_weights.append(int(row['n_visitor']))
            
    bar_race_ratio_sum = bar_black_ratio + bar_white_ratio + bar_asian_ratio + bar_native_ratio        
    if bar_n_visitor_weight != 0 and bar_race_ratio_sum != 0:
        bar_black_ratio /= bar_n_visitor_weight 
        bar_white_ratio /= bar_n_visitor_weight 
        bar_asian_ratio /= bar_n_visitor_weight 
        bar_native_ratio /= bar_n_visitor_weight 
        
        bar_mhi_mean /= bar_n_visitor_weight
        
        weighted_var = np.sum(np.array(bar_weights) * (np.array(bar_mhi_tmp) - bar_mhi_mean)**2) / np.sum(bar_weights)
        
        
        bar_black_ratio /= bar_race_ratio_sum
        bar_white_ratio /= bar_race_ratio_sum
        bar_asian_ratio /= bar_race_ratio_sum
        bar_native_ratio /= bar_race_ratio_sum
        race_entropy = entropy([bar_black_ratio,bar_white_ratio,bar_asian_ratio,bar_native_ratio], base=2)
    
    dict_201901[bar] = [race_entropy,bar_mhi_mean,weighted_var]

In [69]:
dict_201901

{'zzy-223@8g7-37t-nbk': [1.2676244205273797,
  57420.333333333336,
  41158510.88888889],
 '22c-222@8g6-yvg-3kf': [1.3524941085329143,
  75855.96842105263,
  1159568124.241108],
 '222-222@8gb-7gv-3qz': [1.0048328536319067, 83425.75, 554219825.6875],
 '222-222@8g7-484-hqz': [1.3707569644498614, 84299.0, 1235665162.8],
 'zzw-222@8gd-qbf-kmk': [0, 0, 0],
 '224-222@8gb-px9-8n5': [0.7756131270858033,
  102196.96470588235,
  535086819.13993084],
 '226-223@8g7-479-xwk': [1.2207045237435137,
  58702.82258064516,
  719209609.774974],
 '22c-222@8g6-t85-qxq': [0, 0, 0],
 '22n-222@8g7-46n-vzz': [1.152428878571341,
  60140.16788321168,
  517377920.57765466],
 'zzy-22f@8gd-q2q-8n5': [1.0114459399802365,
  79626.14285714286,
  350413084.1224489],
 '22j-224@8g8-7v5-pd9': [0, 0, 0],
 'zzy-222@8gp-ybv-d7q': [0, 0, 0],
 '225-222@8g7-474-5cq': [0.5831323287175088, 125375.0, 0.0],
 '22b-222@8g6-xmy-7yv': [1.0574084111464763,
  74065.46666666666,
  579605159.3155556],
 '222-222@8g6-ym8-dgk': [1.3109744169828

In [76]:
for key in dict_201901:
    if key not in bar_filter:
        del dict_201901[key]

RuntimeError: dictionary changed size during iteration

In [79]:
list(dict_201901.keys())

['zzy-223@8g7-37t-nbk',
 '22c-222@8g6-yvg-3kf',
 '222-222@8gb-7gv-3qz',
 '222-222@8g7-484-hqz',
 '224-222@8gb-px9-8n5',
 '226-223@8g7-479-xwk',
 '22c-222@8g6-t85-qxq',
 '22n-222@8g7-46n-vzz',
 'zzy-22f@8gd-q2q-8n5',
 '22j-224@8g8-7v5-pd9',
 'zzy-222@8gp-ybv-d7q',
 '225-222@8g7-474-5cq',
 '22b-222@8g6-xmy-7yv',
 '222-222@8g6-ym8-dgk',
 '223-223@8g6-ytk-n89',
 '224-222@8g8-7pz-5s5',
 '223-222@8g6-ykz-q4v',
 '22c-222@8g6-t85-r6k',
 'zzw-225@8g7-46n-cnq',
 'zzw-222@8g7-47b-g8v',
 '225-223@8g7-njj-2rk',
 'zzw-223@8g6-y3h-z2k',
 '222-222@8g6-ykz-9mk',
 '222-223@8g6-yjj-8jv',
 '227-222@8gb-rqj-3nq',
 '228-222@8g7-njq-w6k',
 '222-225@8g7-484-hqz',
 '225-222@8g6-y4q-8d9',
 '222-224@8gq-2zf-dn5',
 '229-222@8gj-g7k-vvf',
 'zzy-222@8gd-q5y-6x5',
 '224-222@8g7-bf8-5vf',
 '227-222@8gj-g6r-z75',
 '223-222@8g7-478-p7q',
 '287-222@8g7-46n-8vz',
 '222-224@8gm-kh8-g6k',
 'zzw-224@8g6-ym3-x89',
 'zzw-222@8g7-2f8-xt9',
 'zzw-222@8g8-4g5-h5z',
 '222-222@8g6-t5m-nt9',
 'zzw-222@8g6-ykz-9zz',
 '223-223@8g7-4h

In [80]:
result = [x for x in list(dict_201901.keys()) if x not in bar_filter]

In [83]:
for i in result:
    del dict_201901[i]

In [85]:
len(dict_201901)

589